In [81]:
#Rus first time to imporrt all files
#import nltk
#nltk.download('all')

In [527]:
raw_datatypes = {"integer":"int", "int":"int",
                 "float":"float", "number":"float", "num":"float",
                 "double":"double",
                 "boolean":"bool", "bool":"bool",
                 "character":"char", "char":"char",
                 "string":"str", "str":"str",
                 "void":"void",
                 "array":"array", "arr":"array", "list":"array", "vector":"array"}

datatypes = stemming_dict(raw_datatypes)
#print(datatypes)
#print(datatypes.get('double',0))

raw_operators = {"add":"+", "addition":"+", "+":"+",
                 "sub":"-", "subtract":"-", "subtraction":"-", "-":"-", "minus":"-",
                 "mul":"*", "multiply":"*", "*":"*",
                 "divide":"/","div":"/", "/":"/",
                 "mod":"%", "modulo":"%", "%":"%"}

operators = stemming_dict(raw_operators)
#print(operators)

raw_commands=['create', 'define', 'declare', 'set']

raw_loops=['loop','loops'
           'for loop', 'for loops','for',
           'while loop','while loops', 'while',
           'do while loop','do while loops', 'do while']

custom_stoppers=['\. ',' then ', ' than ']

def extract_words_from_dict():
    
    list_list = [custom_stoppers,raw_commands,raw_loops]
    dict_list = [raw_datatypes, raw_operators]
    
    vocab=set()
    for lists in list_list:
        for inner_list in lists:
            vocab.add(inner_list)
    for dicti in dict_list:
        for keys in dicti.keys():
            vocab.add(keys)
    return vocab

full_vocab = extract_words_from_dict()
#print(full_vocab)

In [528]:
#Tokenize

#from nltk import word_tokenize 
#from nltk.tokenize import sent_tokenize, word_tokenize 

import re

def tokenize1(statement1):   
    return (word_tokenize(statement1))
    '''
    # instantiate tokenizer class
    tokenizer = TweetTokenizer(preserve_case=False, strip_handles=True,
                                   reduce_len=True)
    # tokenize tweets
    tweet_tokens = tokenizer.tokenize(statement)
    return tweet_tokens
    '''
'''
def tokenizer_custom_line(statement):
    tokenize_statment = []
    #tokenize_statment = re.findall(r'[(\w )]* ',statement)
    tokenize_statment = re.split(r"\. |\, ", statement)
    #print (tokenize_statment)
    return (tokenize_statment)

def tokenizer_custom_words(statement):
    tokenize_statement = []
    tokenize_statment = re.split(r" ", statement)
    return(tokenize_statment)
'''

def tokenizer_custom_line_stopper(statement):
    tokenize_statement = []
    
    raw_string=""
    for index, stopper in enumerate(custom_stoppers): #'custom_stoppers' declareed above
        if index==0:
            raw_string = stopper
        else:
            raw_string = raw_string + '|' + stopper
        
    #print("RAW STRING:")
    #print("["+raw_string+"]")
    tokenize_statment = re.split(r""+raw_string, statement)
    return (tokenize_statment)

def tokenizer_custom_comma(statement):   
    if ", " in statement:
        #Find ", "
        statement = statement.replace(', ', ',')
        
    if "," in statement:
        #Find ","
        comma_word = re.findall(r"(,[^\s]+)", statement)
        #print("Comma Word : ",set(comma_word))
       
        temp_list = []
        s=[]
        e=[]
        s.append(0)
        e.append(0)
        index=1
        for each_word in comma_word:
            
            if each_word in [','+vocab for vocab in full_vocab]:
                #print (f"Each Word: {each_word}")
                
                for match in re.finditer(each_word, statement):
                    s.append(match.start())
                    e.append(match.end())
                    #print (f"Index[{index}] is from :{s[index]} {e[index]}")    
                    if (index==1):
                        temp_list.append(statement[s[index-1]:s[index]])
                    else:
                        temp_list.append(statement[s[index-1]+1:s[index]])
                    index +=1

        if index !=1:
            #print(statement[s[i+1]+1:])
            temp_list.append(statement[s[index-1]+1:])  
        #print (f"temp_list: {temp_list}")
        if temp_list:
            return (temp_list)         
    #print (statement)

    #statement = [re.sub(r"(, |,)", r" ", statement)]
    return (statement)
        
    
def remove_extra_space(statement):
    while "  " in statement:
        statement = statement.replace("  ", " ")
    return (statement)

def tokenize_sentence_advance(statement):
    statement = remove_extra_space(statement)
    statement = statement.lower()
    
    new_statement = tokenizer_custom_line_stopper(statement)
    #print (new_statement)
    
    #temp_list=[]
    final_statement=[]
    for each_line in new_statement:
        temp_list = tokenizer_custom_comma(each_line)
        #print ("Temp_list:",temp_list)
        if temp_list != each_line :
            for each_line in temp_list:
                final_statement.append(each_line)
        else:
            final_statement.append(each_line)
    return (final_statement)

#Testing
'''
#custom_statement_raw = "define Variable a,b and 'c'. Add variables a,c and b, Subtract variables b from a then create a for loop and multiply a 5 times"
#custom_statement_raw = "define Variable a and 'b'. Add variables a from b, Subtract variables b from a, create a for loop and multiply a 5 times then add 2 to a"
#custom_statement_raw = "Define a and b then add 1 and 2 then create a for loop"
#custom_statement_raw = "Define a and b,"
print(custom_statement_raw)
print(tokenize_sentence_advance(custom_statement_raw))

custom_statement_raw = remove_extra_space(custom_statement_raw)
cust_statement1 = tokenizer_custom_line_stopper(custom_statement_raw)
print(cust_statement1)
for each_line in cust_statement1:
    tokenizer_custom_comma(each_line)
    #print (tokenizer_custom_words(each_line))
'''
print("",end="")

In [529]:
#Removing Stop Words

from nltk.corpus import stopwords 

stop_words = set(stopwords.words('english')) 
filtered_sentence = [[]] *len(tokenize_sentence)

def remove_stopwords(sentence):
    stop_words = set(stopwords.words('english'))
    filtered_sentence = []
    for each_word in sentence:
        if each_word not in stop_words:
            filtered_sentence.append(each_word)
    return (filtered_sentence)


In [530]:
#Stemming
from nltk.stem import PorterStemmer

def stemming_list(statement1):
    # Instantiate stemming class
    stemmer = PorterStemmer() 

    # Create an empty list to store the stems
    sentence_stem = [] 

    for word in statement1:
        stem_word = stemmer.stem(word)  # stemming word
        sentence_stem.append(stem_word)  # append to the list

    return(sentence_stem)

def stemming_dict(dicti):
    stemmer = PorterStemmer()
    
    sentence_stem={}
    for word, value in dicti.items():
        stem_word = stemmer.stem(word)
        sentence_stem[stem_word] = value
    return sentence_stem

In [531]:
#Main

#custom_statement_raw = "define Variable a and 'b'. Add variables a from b, Subtract variables b from a, create a for loop and multiply a 5 times"
custom_statement_raw = "define Variable var1 and 'var2'. Add variables var1 from var2, Subtract variables var2 from var1, create a for loop and multiply var1 5 times"
print(custom_statement_raw)

'''modified_statement = custom_statement_raw.lower()
modified_statement = tokenizer_custom_line(modified_statement)  
modified_statement2 = []
for each_line in modified_statement:
    modified_statement2.append(tokenizer_custom_words(each_line))
print("\nAfter Tokenizer:")
print(modified_statement2)
                               
#modified_statement = stemming_list(modified_statement)
print("\nAfter Porter Stemming:")
#print(modified_statement)
modified_statement3 = []
for each_line in modified_statement2:
    modified_statement3.append(stemming_list(each_line))
print(modified_statement3)
'''
tokenize_sentence = tokenize_sentence_advance(custom_statement_raw)
print("\nAfter Tokenization of Each Line")
print(tokenize_sentence)

tokenize_words=[]
for each_line in tokenize_sentence:
    tokenize_words.append(tokenizer_custom_words(each_line))
print("\nAfter Tokenization of Each Word")
print(tokenize_words)


filtered_sentence=[]
for each_line in tokenize_words:
    filtered_sentence.append(remove_stopwords(each_line) )
print("\nRemoved Stopwords:")
print(filtered_sentence)

define Variable var1 and 'var2'. Add variables var1 from var2, Subtract variables var2 from var1, create a for loop and multiply var1 5 times

After Tokenization of Each Line
["define variable var1 and 'var2'", 'add variables var1 from var2', 'subtract variables var2 from var1', 'create a for loop and multiply var1 5 times']

After Tokenization of Each Word
[['define', 'variable', 'var1', 'and', "'var2'"], ['add', 'variables', 'var1', 'from', 'var2'], ['subtract', 'variables', 'var2', 'from', 'var1'], ['create', 'a', 'for', 'loop', 'and', 'multiply', 'var1', '5', 'times']]

Removed Stopwords:
[['define', 'variable', 'var1', "'var2'"], ['add', 'variables', 'var1', 'var2'], ['subtract', 'variables', 'var2', 'var1'], ['create', 'loop', 'multiply', 'var1', '5', 'times']]
